## Imports

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pickle
from bigram import BigramLM
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from utils import emotion_scores
import warnings
warnings.filterwarnings('ignore')

C:\Users\Deeptanshu Barman\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


## Model Setup

In [76]:
corpus_file_path = '../data/corpus.txt'

# Read the content of the file
with open(corpus_file_path, 'r') as file:
    lines = file.readlines()

# Add a "$" sign to the beginning of each line
modified_lines = ['$ ' + line.strip() for line in lines]
joined_string = '\n'.join(modified_lines)

In [77]:
corpus=joined_string

In [113]:
model = BigramLM(corpus)
model.set_token()
model.calculate_bigrams()
laplace_matrix = model.get_laplace_matrix()
kn_matrix = model.set_kn_matrix(d=0.75)

Model Initialized 🟢
Tokens Set 🟢


Populating Bigram Matrix...: 100%|██████████| 48931/48931 [00:00<00:00, 438751.01it/s]


All Matrices Calculated 🟢


Calculating Kneser-Ney Matrix...: 100%|██████████| 5430/5430 [00:56<00:00, 96.12it/s] 


In [104]:
temp=model.get_normal_matrix()
for i in range(5430):
    if temp[i,0]==1:
        print(model.get_token()[i])

abilities
abuses
abusive
accent
ached
acts
advent
afaerytaleinmakebelieve
aging
agreement
ambition
american
amusement
anguished
aoi
apartment
appealing
appearance
appetite
appts
arena
artwork
assessment
assistant
athletics
ativan
attending
attitude
attitudes
awards
awareness
awhile
ba
baffroom
bags
baking
ballistic
barrage
batch
begleiter
behaviors
blankets
blessing
bloke
blue
bondmusings
boob
bookmark
boots
breastfeeding
bridge
bull
buys
capp
cara
category
challenged
chapters
chiangmai
china
christians
chuffed
circle
clara
closet
clover
clues
clutter
collected
column
comings
commented
completing
concept
concieve
conclusions
continuously
cookie
cooperation
coppers
corinthians
cousins
coworker
crew
crone
cynical
denmark
description
deserted
design
desp
deteriorate
devices
disillusioned
disoriented
displaced
display
disregarded
dissolve
dlpt
doll
dope
dr
dressed
dvd
easier
east
easygoing
eaters
education
effort
eh
eid
emergence
empathy
empowered
emptied
entertaining
erik
essence
etsy
eve

In [95]:
model.get_token()

['$',
 'a',
 'aa',
 'aahhh',
 'abandoning',
 'abilities',
 'ability',
 'abit',
 'able',
 'abound',
 'about',
 'above',
 'absolute',
 'absolutely',
 'abt',
 'abundance',
 'abused',
 'abuses',
 'abusive',
 'abyss',
 'academic',
 'academics',
 'accelerated',
 'accent',
 'accept',
 'acceptable',
 'acceptance',
 'accepted',
 'accepting',
 'accessaries',
 'accessibility',
 'accident',
 'accidentally',
 'accomplished',
 'accomplishing',
 'accordance',
 'according',
 'account',
 'accusing',
 'accustomed',
 'ace',
 'ache',
 'ached',
 'achieve',
 'achieved',
 'achieving',
 'aching',
 'acne',
 'acquainted',
 'acronym',
 'across',
 'act',
 'action',
 'actions',
 'active',
 'actively',
 'activist',
 'activities',
 'activity',
 'acts',
 'actual',
 'actually',
 'ad',
 'adams',
 'adapt',
 'add',
 'added',
 'addiction',
 'addictive',
 'adjust',
 'administration',
 'admiration',
 'admire',
 'admired',
 'admiring',
 'admit',
 'admitting',
 'adn',
 'adolescence',
 'adomen',
 'adopt',
 'adore',
 'adoring',

## Generating Emotion for all non-zero Bigrams

In [80]:
tokens = model.get_token()
count_matrix = model.get_count_matrix()

In [81]:
tokens

['$',
 'a',
 'aa',
 'aahhh',
 'abandoning',
 'abilities',
 'ability',
 'abit',
 'able',
 'abound',
 'about',
 'above',
 'absolute',
 'absolutely',
 'abt',
 'abundance',
 'abused',
 'abuses',
 'abusive',
 'abyss',
 'academic',
 'academics',
 'accelerated',
 'accent',
 'accept',
 'acceptable',
 'acceptance',
 'accepted',
 'accepting',
 'accessaries',
 'accessibility',
 'accident',
 'accidentally',
 'accomplished',
 'accomplishing',
 'accordance',
 'according',
 'account',
 'accusing',
 'accustomed',
 'ace',
 'ache',
 'ached',
 'achieve',
 'achieved',
 'achieving',
 'aching',
 'acne',
 'acquainted',
 'acronym',
 'across',
 'act',
 'action',
 'actions',
 'active',
 'actively',
 'activist',
 'activities',
 'activity',
 'acts',
 'actual',
 'actually',
 'ad',
 'adams',
 'adapt',
 'add',
 'added',
 'addiction',
 'addictive',
 'adjust',
 'administration',
 'admiration',
 'admire',
 'admired',
 'admiring',
 'admit',
 'admitting',
 'adn',
 'adolescence',
 'adomen',
 'adopt',
 'adore',
 'adoring',

In [29]:
emotions = {}
for i, token in tqdm(enumerate(tokens)):
    for j, token2 in enumerate(tokens):
        if count_matrix[i][j] > 0 and token!='$' and token2!="$":
            emotions[(token, token2)] = emotion_scores(str(token + " " + token2))
        if count_matrix[i][j] > 0 and token=='$' and token2!="$":
            emotions[(token, token2)]= emotion_scores(str(token2))
        if count_matrix[i][j] > 0 and token!='$' and token2=="$":
            emotions[(token, token2)]= emotion_scores(str(token))

with open('emotions.pkl', 'wb') as f:
    pickle.dump(emotions, f)

5430it [12:24,  7.29it/s]


In [71]:
emo = pickle.load(open("pickle_files/emotions.pkl", "rb"))

## Generating 50 Sentences each of every emotion


In [131]:
emotions=["sadness", "joy", "love", "anger", "fear", "surprise"]
for i in emotions:
    # used count matrix use some other matrix like laplace or kneser-ney
    model.generate_sentences(model.get_laplace_matrix(), emotion=i, word_limit=15, no_of_sentences=50, alpha = 1, beta = 0.1)

Generating sadness Matix: 5430it [00:19, 283.16it/s]
Generating Sentence: 100%|██████████| 50/50 [00:01<00:00, 27.18it/s]


Sentences Generated + Stored 🟢


Generating joy Matix: 5430it [00:22, 246.13it/s]
Generating Sentence: 100%|██████████| 50/50 [00:02<00:00, 23.10it/s]


Sentences Generated + Stored 🟢


Generating love Matix: 5430it [00:21, 256.38it/s]
Generating Sentence: 100%|██████████| 50/50 [00:02<00:00, 22.42it/s]


...

Sentences Generated + Stored 🟢


Generating fear Matix: 5430it [00:20, 259.83it/s]
Generating Sentence: 100%|██████████| 50/50 [00:02<00:00, 22.59it/s]


Sentences Generated + Stored 🟢


Generating surprise Matix: 5430it [00:21, 249.52it/s]
Generating Sentence: 100%|██████████| 50/50 [00:02<00:00, 23.28it/s]

Sentences Generated + Stored 🟢


## Training SVC Classifier

In [35]:
corpus_file = "..\data\corpus.txt"
labels_file = "..\data\labels.txt"
with open(corpus_file) as f:
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(f)
    fts = vectorizer.get_feature_names_out()
# print(X)

with open(labels_file) as l:
    y = l.read().splitlines()

In [109]:
parameters = {'kernel': ('linear', 'rbf'), 'C': [1, 3, 5], 'gamma': [0.1, 0.05, 0.01], "degree": [2, 3, 5], 'tol': [0.001, 0.0001]}
# clf = SVC(C=1.0, kernel='rbf', degree=3, gamma='scale')
clf = GridSearchCV(SVC(), parameters, n_jobs=4)
clf.fit(X, y)
best_model=clf.best_estimator_
print(clf.best_params_)

{'C': 1, 'degree': 2, 'gamma': 0.1, 'kernel': 'linear', 'tol': 0.0001}


In [146]:
# print(clf.best_params_)
# print(clf.best_score_)

In [110]:
print(best_model.score(X, y))

0.9858333333333333


In [132]:
emotions=["sadness", "joy", "love", "anger", "fear", "surprise"]
dataset = {}
for emotion in emotions:
    path = ".\emotion_text\gen_"+emotion+".txt"
    with open(path, 'r') as file:
        dataset[emotion] = file.read().splitlines()

In [133]:
def emotion_accuracy(emotion, dataset, best_model):
    X = vectorizer.transform(dataset[emotion])
    y = [emotion]*len(dataset[emotion])
    return best_model.score(X, y)

for emotion in emotions:
    print(emotion, emotion_accuracy(emotion, dataset, best_model)) 
   
print("Average accuracy: ", np.mean([emotion_accuracy(emotion, dataset, best_model) for emotion in emotions]))

sadness 0.18
joy 0.6
love 0.18
anger 0.22
fear 0.2
surprise 0.04
Average accuracy:  0.23666666666666666
